In [10]:
import pandas as pd
import numpy as np

In [2]:
sessions = pd.read_csv('./data/processed_sessions.csv')

In [4]:
sessions[sessions["not_skipped"]==False]

,session_id,track_id,not_skipped
5,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_c815228b-3212-4f9e-9d4f-9cb19b248184,False
6,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_e23c19f5-4c32-4557-aa44-81372c2e3705,False
7,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_0be6eced-f56f-48bd-8086-f2e0b760fdee,False
8,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_f3ecbd3b-9e8e-4557-b8e0-39cfcd7e65dd,False
9,0_00006f66-33e5-4de7-a324-2d18e439fc1e,t_2af4dfa0-7df3-4b7e-b7ab-353ba48237f9,False
...,...,...,...
167856,0_0eae096d-ae4c-4556-8227-0f6294958330,t_d4c83551-0b13-4fe0-bca6-7392f5f7292f,False
167857,0_0eae096d-ae4c-4556-8227-0f6294958330,t_8160da16-8962-4f26-8e3b-ea29377415f2,False
167858,0_0eae096d-ae4c-4556-8227-0f6294958330,t_56bc277a-86e3-4a74-8cf4-93a45f7fc4d6,False
167859,0_0eae096d-ae4c-4556-8227-0f6294958330,t_d8c9417d-d0f5-40df-82d4-c3298bbafc91,False


In [12]:
sessions["not_skipped"] = sessions["not_skipped"].apply(lambda x: float(x))

In [16]:
sessions_array = np.array(sessions)
sessions_array

array([['0_00006f66-33e5-4de7-a324-2d18e439fc1e',
        't_0479f24c-27d2-46d6-a00c-7ec928f2b539', 1.0],
       ['0_00006f66-33e5-4de7-a324-2d18e439fc1e',
        't_9099cd7b-c238-47b7-9381-f23f2c1d1043', 1.0],
       ['0_00006f66-33e5-4de7-a324-2d18e439fc1e',
        't_fc5df5ba-5396-49a7-8b29-35d0d28249e0', 1.0],
       ...,
       ['0_0eaeef5d-25e9-4429-bd55-af15d3604c9f',
        't_f673e1b7-4ebe-4fc1-ac24-a9f25de70381', 1.0],
       ['0_0eaeef5d-25e9-4429-bd55-af15d3604c9f',
        't_e172e8e7-7161-42a9-acb0-d606346c8f87', 1.0],
       ['0_0eaeef5d-25e9-4429-bd55-af15d3604c9f',
        't_77977dd6-597e-4425-8f8f-4efb32ecfba6', 1.0]], dtype=object)

In [9]:
import cornac
from cornac.eval_methods import RatioSplit

FM model is only supported on Linux.
Windows executable can be found at http://www.libfm.org.


In [38]:
ratio_split = RatioSplit(
    data=sessions_array,
    test_size=0.2,
    rating_threshold=1.0,
    seed=123,
    exclude_unknowns=True,
    verbose=True,
)

rating_threshold = 1.0
exclude_unknowns = True


c:\users\alons\appdata\local\programs\python\python38\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 14686 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


---
Training data:
Number of users = 10000
Number of items = 43979
Number of ratings = 119618
Max rating = 1.0
Min rating = 0.0
Global mean = 0.3
---
Test data:
Number of users = 8838
Number of items = 9792
Number of ratings = 25412
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 10000
Total items = 43979


c:\users\alons\appdata\local\programs\python\python38\lib\site-packages\cornac\data\dataset.py:361: UserWarning: 1019 duplicated observations are removed!
  warnings.warn("%d duplicated observations are removed!" % dup_count)


In [36]:
gmf = cornac.models.GMF(
    num_factors=8,
    num_epochs=10,
    learner="adam",
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
mlp = cornac.models.MLP(
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf1 = cornac.models.NeuMF(
    num_factors=8,
    layers=[64, 32, 16, 8],
    act_fn="tanh",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
)
neumf2 = cornac.models.NeuMF(
    name="NeuMF_pretrained",
    learner="adam",
    num_epochs=10,
    batch_size=256,
    lr=0.001,
    num_neg=50,
    seed=123,
    num_factors=gmf.num_factors,
    layers=mlp.layers,
    act_fn=mlp.act_fn,
).pretrain(gmf, mlp)

# Instantiate evaluation metrics
ndcg_10 = cornac.metrics.NDCG(k=10)
MAP = cornac.metrics.MAP()

# Put everything together into an experiment and run it
cornac.Experiment(
    eval_method=ratio_split,
    models=[gmf, mlp, neumf1, neumf2],
    metrics=[ndcg_10, MAP],
).run()


[GMF] Training started!


AttributeError: module 'tensorflow' has no attribute 'variable_scope'

In [41]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, MostPop, SVD, WBPR
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP

# initialize models
models = [
    MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123),
    PMF(k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=123),
    BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123),
    MostPop(),
    WBPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.001, seed=123),
    SVD(k=10, max_iter=30, learning_rate=0.01, lambda_reg=0.02, seed=123)
    
]

# define metrics to evaluate the models
metrics = [NDCG(k=10), MAP()]

# put it together in an experiment, voilà!
cornac.Experiment(eval_method=ratio_split, models=models, metrics=metrics, user_based=True).run()+++++++++++


[MF] Training started!

[MF] Evaluation started!


Ranking:   0%|          | 0/8838 [00:00<?, ?it/s]


[PMF] Training started!

[PMF] Evaluation started!


Ranking:   0%|          | 0/8838 [00:00<?, ?it/s]


[BPR] Training started!

[BPR] Evaluation started!


...


[WBPR] Training started!

[WBPR] Evaluation started!


Ranking:   0%|          | 0/8838 [00:00<?, ?it/s]


[SVD] Training started!

[SVD] Evaluation started!


Ranking:   0%|          | 0/8838 [00:00<?, ?it/s]


TEST:
...
        |    MAP | NDCG@10 | Train (s) | Test (s)
------- + ------ + ------- + --------- + --------
MF      | 0.0008 |  0.0008 |    0.4661 |  81.2652
PMF     | 0.0118 |  0.0136 |   20.8670 |  91.5825
BPR     | 0.0343 |  0.0366 |   11.3920 |  81.5650
MostPop | 0.0342 |  0.0368 |    0.0130 |  31.9570
WBPR    | 0.0253 |  0.0320 |   20.0931 |  91.1635
SVD     | 0.0007 |  0.0008 |    0.4670 |  87.0430

